<a href="https://colab.research.google.com/github/DmitryKutsev/ml_hw/blob/master/hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
!wget https://raw.githubusercontent.com/DmitryKutsev/ml_hw/master/small_data.csv


--2020-03-04 14:20:28--  https://raw.githubusercontent.com/DmitryKutsev/ml_hw/master/small_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24394136 (23M) [text/plain]
Saving to: ‘small_data.csv.2’

small_data.csv.2    100%[===================>]  23.26M   113MB/s    in 0.2s    

2020-03-04 14:20:28 (113 MB/s) - ‘small_data.csv.2’ saved [24394136/24394136]



In [0]:
import pandas as pd
from sklearn import cluster
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import AffinityPropagation, MeanShift, AgglomerativeClustering
from sklearn.decomposition import TruncatedSVD

In [0]:

from sklearn.cluster import AffinityPropagation, AgglomerativeClustering, DBSCAN, \
                            KMeans, MiniBatchKMeans, Birch, MeanShift, SpectralClustering
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, \
                            silhouette_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import metrics
from sklearn.datasets import make_blobs

In [0]:
data = pd.read_csv('small_data.csv')

In [107]:
data.head(5)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1
0,793dd3d5f0f0,9467e0341c0e,Алтайский край,Барнаул,Личные вещи,Детская одежда и обувь,Для девочек,Платья и юбки,146-152 см (10-12 лет),Платье праздничное для девочки,Очень красивое торг,1500.0,106,2017-04-12,Company,ace7b2b7f066a6b6ff244f90d030c2c1a1bf118dd0e1ac...,573.0
1,f8c965d9b668,1905516660f5,Свердловская область,Каменск-Уральский,Для дома и дачи,Мебель и интерьер,"Кровати, диваны и кресла",NaN,NaN,Кровать двухспальная,Двуспальная кровать 140х200 с металлическим ка...,5700.0,10,2017-04-15,Private,b38ef24dd5b876ed626bd98241e228e4e19e4160d33abe...,1439.0
2,2f153c00d561,b4dc4849fdd4,Самарская область,Самара,Бытовая электроника,Телефоны,Samsung,NaN,NaN,Samsung S3 mini gt-i8190,"Продам телефон самсунг 2 ядра, 1 гиг оперативн...",1500.0,6,2017-04-12,Private,8530cc0c8d5dc6d212497dffd610d89b92a7db952934db...,2932.0
3,7fc5eea56915,02c14d39fdb4,Саратовская область,Саратов,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,44–46 (M),"Красивое платье на одно плечо, в идеале размер м","Заказывала в Сша, прекрасное качество, в деле ...",1000.0,1280,2017-04-13,Company,eb21745ba276d3b00b05856a58fe719e31a82419c0221a...,386.0
4,b01000d29ea3,b17f14110367,Удмуртия,Ижевск,Личные вещи,Детская одежда и обувь,Для мальчиков,"Шапки, варежки, шарфы",NaN,Стильная панама,Состояние новой. Размер до года- полтора. По ...,150.0,145,2017-04-14,Company,5bf1001fa681b96f121ad8047c85e3747f3dac37830dd0...,209.0


In [0]:
data = data[['category_name', 'title']]

In [109]:
data.shape

(37339, 2)

In [0]:
sample = data.sample(frac=0.1)

In [111]:
sample.shape

(3734, 2)

In [0]:

cv = CountVectorizer(max_features=500)
X = cv.fit_transform(sample['title'])

y = sample['category_name']

In [113]:
cluster = AffinityPropagation(damping=0.7, preference=-5, 
                              max_iter=400, verbose=2)
cluster.fit(X)

Converged after 272 iterations.


AffinityPropagation(affinity='euclidean', convergence_iter=15, copy=True,
                    damping=0.7, max_iter=400, preference=-5, verbose=2)

In [0]:
labels = cluster.labels_

In [115]:
n_clusters = len(set(labels))
n_clusters

224

In [116]:
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

Silhouette Coefficient: 0.360


In [117]:
# эти три метрки что-то вроде точности, полноты и ф-меры для кластеризации
print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, labels)) # проверяет, что в кластере объекты одного класса
print("Completeness: %0.3f" % metrics.completeness_score(y, labels)) # проверяет, что объекты класса только в одном кластере
print("V-measure: %0.3f" % metrics.v_measure_score(y, labels)) # превращает обе метрики в одну 

print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(y, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(y, labels))

Homogeneity: 0.497
Completeness: 0.380
V-measure: 0.431
Adjusted Rand Index: -0.007
Adjusted Mutual Information: 0.303


In [0]:
sample['cluster'] = cluster.labels_

In [119]:
sample[sample.cluster==39].head(10)

,category_name,title,cluster
27691,Комнаты,"Комната 29.6 м² в 2-к, 1/2 эт.",39
22099,Комнаты,"Комната 13 м² в 1-к, 5/5 эт.",39
5473,Комнаты,"Комната 13 м² в 4-к, 4/5 эт.",39
18523,Комнаты,"Комната 13 м² в 1-к, 5/5 эт.",39
19147,Комнаты,"Комната 13 м² в 1-к, 4/5 эт.",39


In [120]:
sample[sample.category_name=='Телефоны'].cluster.value_counts()

0      43
213    23
90      8
34      8
112     8
107     7
68      7
73      6
158     5
2       5
176     5
62      4
19      4
5       3
193     2
100     1
96      1
118     1
140     1
7       1
Name: cluster, dtype: int64

In [0]:
#MS тоже не умеет работать с sparse матрицами

In [0]:
cv = CountVectorizer(max_features=2000)
svd = TruncatedSVD(50)
X = cv.fit_transform(sample['title'])
X_svd = svd.fit_transform(X)

y = sample['category_name']

In [123]:
cluster.estimate_bandwidth(X_svd)

AttributeError: ignored

In [0]:
# cluster_all=False чтобы не записывать все объекты в кластеры
cluster = MeanShift(cluster_all=False, bandwidth=0.5)
cluster.fit(X_svd)

In [0]:
sample = data.sample(frac=0.5)

In [0]:
sample.shape

In [0]:

cv = CountVectorizer(max_features=500)
svd = TruncatedSVD(50)
X = cv.fit_transform(sample['title'])
X_svd = svd.fit_transform(X)

y = sample['category_name']

In [0]:
len(y.unique())

In [0]:
X.shape

In [124]:
import sklearn
sklearn.cluster.estimate_bandwidth(X_svd)

0.7324824749735146

In [125]:
# cluster_all=False чтобы не записывать все объекты в кластеры
cluster = MeanShift(cluster_all=False, bandwidth=0.5)
cluster.fit(X_svd)

MeanShift(bandwidth=0.5, bin_seeding=False, cluster_all=False, max_iter=300,
          min_bin_freq=1, n_jobs=None, seeds=None)

In [126]:
len(set(cluster.labels_))

346

In [0]:
labels = cluster.labels_

In [128]:
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

Silhouette Coefficient: 0.056


In [129]:
# эти три метрки что-то вроде точности, полноты и ф-меры для кластеризации
print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, labels)) # проверяет, что в кластере объекты одного класса
print("Completeness: %0.3f" % metrics.completeness_score(y, labels)) # проверяет, что объекты класса только в одном кластере
print("V-measure: %0.3f" % metrics.v_measure_score(y, labels)) # превращает обе метрики в одну 

print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(y, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(y, labels))

Homogeneity: 0.376
Completeness: 0.363
V-measure: 0.369
Adjusted Rand Index: -0.014
Adjusted Mutual Information: 0.236


In [0]:
sample['cluster'] = cluster.labels_

In [132]:
sample[sample.cluster==2].head()

,category_name,title,cluster
20877,Товары для детей и игрушки,Стерилизатор для бутылочек Philips Avent 3 в 1,2
20020,Оборудование для бизнеса,Помпа WW186 для Elit2840 TX/TSX 13,2
5382,Планшеты и электронные книги,Тачскрин ZJ-80032A для планшета BQ-8005G,2
23649,Мебель и интерьер,Рамка для фото,2
12586,Красота и здоровье,Шарики для стерилизатора,2


In [0]:
sample = data.sample(frac=0.01)

In [134]:
sample.shape

(373, 2)

In [0]:
cv = CountVectorizer(max_features=500)
svd = TruncatedSVD(50)
X = cv.fit_transform(sample['title'])
X_svd = svd.fit_transform(X)

y = sample['category_name']

In [141]:
cluster = SpectralClustering(n_clusters=47)
cluster.fit(X_svd)

SpectralClustering(affinity='rbf', assign_labels='kmeans', coef0=1, degree=3,
                   eigen_solver=None, eigen_tol=0.0, gamma=1.0,
                   kernel_params=None, n_clusters=47, n_components=None,
                   n_init=10, n_jobs=None, n_neighbors=10, random_state=None)

In [142]:
X.shape

(373, 500)

In [143]:
labels = cluster.labels_
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

Silhouette Coefficient: 0.083


In [144]:
print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, labels)) # проверяет, что в кластере объекты одного класса
print("Completeness: %0.3f" % metrics.completeness_score(y, labels)) # проверяет, что объекты класса только в одном кластере
print("V-measure: %0.3f" % metrics.v_measure_score(y, labels)) # превращает обе метрики в одну 

print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(y, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(y, labels))

Homogeneity: 0.329
Completeness: 0.476
V-measure: 0.389
Adjusted Rand Index: 0.005
Adjusted Mutual Information: 0.157


In [0]:
sample['cluster'] = cluster.labels_

In [152]:
sample[sample.cluster==17]

,category_name,title,cluster
1026,Спорт и отдых,"Комплект сноубордический (сноуборд, ботинки, к...",17


In [0]:
sample = data.sample(frac=0.5)

In [154]:
sample.shape

(18670, 2)

In [0]:
cv = CountVectorizer(max_features=2000)
svd = TruncatedSVD(50)
X = cv.fit_transform(sample['title'])
X_svd = svd.fit_transform(X)

y = sample['category_name']

In [156]:
X.shape

(18670, 2000)

In [157]:

# eps - задает максимальную для объединения дистанцию между объектами == размер окрестности
# min_samples - минимальное количество объектов в классе

cluster = DBSCAN(min_samples=10, eps=0.3) 
cluster.fit(X_svd)

DBSCAN(algorithm='auto', eps=0.3, leaf_size=30, metric='euclidean',
       metric_params=None, min_samples=10, n_jobs=None, p=None)

In [0]:
sample['cluster'] = cluster.labels_

In [163]:
sample[sample.cluster==1].head()

,category_name,title,cluster
10440,Охота и рыбалка,Стойка подставка держатель для удочки,1
20339,Красота и здоровье,Гидромасажная ванна для ног,1
22456,Телефоны,Чехлы для телефонов,1
5044,Товары для компьютера,DDR3L 2gb для ноутбука,1
29003,Спорт и отдых,Фильтр для очистки воды,1


In [0]:
sample = data

In [174]:
sample.shape

(37339, 2)

In [0]:
cv = CountVectorizer(max_features=500)
svd = TruncatedSVD(50)
X = cv.fit_transform(sample['title'])
X_svd = svd.fit_transform(X)

y = sample['category_name']

In [176]:
X.shape

(37339, 500)

In [177]:

cluster = KMeans(n_clusters=47)
cluster.fit(X_svd)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=47, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [0]:
labels = cluster.labels_

In [179]:
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X[:10000], labels[:10000]))

Silhouette Coefficient: 0.240


In [180]:
print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, labels)) # проверяет, что в кластере объекты одного класса
print("Completeness: %0.3f" % metrics.completeness_score(y, labels)) # проверяет, что объекты класса только в одном кластере
print("V-measure: %0.3f" % metrics.v_measure_score(y, labels)) # превращает обе метрики в одну 

print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(y, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(y, labels))

Homogeneity: 0.293
Completeness: 0.376
V-measure: 0.330
Adjusted Rand Index: -0.003
Adjusted Mutual Information: 0.322


In [0]:
sample['cluster'] = cluster.labels_

In [182]:
sample[sample.cluster==13].head()

,category_name,title,cluster
71,Детская одежда и обувь,Комбинезон джинсовый,13
123,Детская одежда и обувь,Комбинезон-трансформер,13
154,Детская одежда и обувь,Комбинезон,13
321,Детская одежда и обувь,Комбинезон,13
331,Детская одежда и обувь,Комбинезон Arctiline,13


In [0]:
#MiniBatchKMeans

In [0]:
cv = CountVectorizer(max_features=5000)
svd = TruncatedSVD(200)
X = cv.fit_transform(data['title'])
X_svd = svd.fit_transform(X)

y = data['category_name']

In [185]:
X.shape

(37339, 5000)

In [186]:
cluster = MiniBatchKMeans(n_clusters=1000, init_size=5000, verbose=1, max_iter=5000, 
                          max_no_improvement=100, reassignment_ratio=0.3)
cluster.fit(X_svd)

Init 1/3 with method: k-means++
Inertia for init 1/3: 732.103299
Init 2/3 with method: k-means++
Inertia for init 2/3: 736.306864
Init 3/3 with method: k-means++
Inertia for init 3/3: 739.362718
Minibatch iteration 1/1870000: mean batch inertia: 0.252925, ewa inertia: 0.252925 
Minibatch iteration 2/1870000: mean batch inertia: 0.236116, ewa inertia: 0.252835 
Minibatch iteration 3/1870000: mean batch inertia: 0.161701, ewa inertia: 0.252346 
Minibatch iteration 4/1870000: mean batch inertia: 0.170488, ewa inertia: 0.251908 
Minibatch iteration 5/1870000: mean batch inertia: 0.182948, ewa inertia: 0.251539 
Minibatch iteration 6/1870000: mean batch inertia: 0.220664, ewa inertia: 0.251373 
Minibatch iteration 7/1870000: mean batch inertia: 0.216140, ewa inertia: 0.251185 
Minibatch iteration 8/1870000: mean batch inertia: 0.220295, ewa inertia: 0.251019 
Minibatch iteration 9/1870000: mean batch inertia: 0.139360, ewa inertia: 0.250421 
[MiniBatchKMeans] Reassigning 50 cluster centers.

MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
                init_size=5000, max_iter=5000, max_no_improvement=100,
                n_clusters=1000, n_init=3, random_state=None,
                reassignment_ratio=0.3, tol=0.0, verbose=1)

In [0]:

labels = cluster.labels_

In [188]:

print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X[:10000], labels[:10000]))

Silhouette Coefficient: 0.098


In [189]:
# эти три метрки что-то вроде точности, полноты и ф-меры для кластеризации
print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, labels)) # проверяет, что в кластере объекты одного класса
print("Completeness: %0.3f" % metrics.completeness_score(y, labels)) # проверяет, что объекты класса только в одном кластере
print("V-measure: %0.3f" % metrics.v_measure_score(y, labels)) # превращает обе метрики в одну 

print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(y, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(y, labels))

Homogeneity: 0.618
Completeness: 0.301
V-measure: 0.405
Adjusted Rand Index: 0.017
Adjusted Mutual Information: 0.344
